<a href="https://colab.research.google.com/github/ndvp39/CloudComputing-tirgul/blob/main/searchEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [192]:
from google.colab import drive, output
from IPython.display import HTML, Javascript
from IPython.display import clear_output
from firebase_admin import credentials
from firebase_admin import firestore
import firebase_admin
import IPython
import random
import json
import os

directory = '/content/CloudComputing-tirgul'

if os.path.exists(directory):
    !rm -r "$directory"

os.chdir('/content')  # Change current directory to /content
!git clone "https://github.com/ndvp39/CloudComputing-tirgul"

drive.mount('/content/drive')

Cloning into 'CloudComputing-tirgul'...
remote: Enumerating objects: 4157, done.
remote: Counting objects: 100% (898/898), done.
remote: Compressing objects: 100% (440/440), done.
remote: Total 4157 (delta 501), reused 779 (delta 435), pack-reused 3259
Receiving objects: 100% (4157/4157), 20.23 MiB | 15.17 MiB/s, done.
Resolving deltas: 100% (1495/1495), done.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [145]:
!pip install requests beautifulsoup4

import requests
from bs4 import BeautifulSoup

import nltk
from nltk.chat.util import Chat, reflections

In [164]:
%cd /content/CloudComputing-tirgul/Project/Index/

%run index.ipynb

/content/CloudComputing-tirgul/Project/Index


Index - main

In [176]:
# GET INDEX FROM DB
index = get_index_from_db()
print(index)

{'triallearn': {'https://www.redhat.com/en/resources': {'id': 7, 'counter': 2}, 'https://www.redhat.com/en': {'counter': 2, 'id': 1}, 'https://www.redhat.com/en/blog?f[0]=taxonomy_topic_tid:9001#rhdc-search-listing': {'id': 17, 'counter': 2}, 'https://www.redhat.com/en/blog?f[0]=taxonomy_topic_tid:75491#rhdc-search-listing': {'id': 16, 'counter': 2}, 'https://www.redhat.com/en/topics': {'id': 27, 'counter': 2}, 'https://www.redhat.com/en/services/training/all-courses-exams?f%5B0%5D=taxonomy_product_tid%3A6921&intcmp=7013a0000034hcuAAA': {'id': 10, 'counter': 2}, 'https://www.redhat.com/en/topics/integration/whats-the-difference-between-soap-rest': {'id': 20, 'counter': 2}, 'https://www.redhat.com/en/success-stories/banco-galicia-NLP': {'id': 13, 'counter': 2}, 'https://www.redhat.com/en/blog?f[0]=taxonomy_topic_tid:27061#rhdc-search-listing': {'id': 22, 'counter': 2}, 'https://www.redhat.com/en/topics/security/api-security': {'id': 25, 'counter': 2}, 'https://www.redhat.com/en/topics/e

ChatBot

In [148]:
%cd /content/CloudComputing-tirgul/Project/Chatbot/
%run chatbot.ipynb

/content/CloudComputing-tirgul/Project/Chatbot
/content/CloudComputing-tirgul/Project/Index


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[('(.*)(.*)it(.*)(.*)', ["DevSecOps stands for development, security, and operations. It's an approach to culture, automation, and platform design that integrates security as a shared responsibility throughout the entire IT lifecycle.you can read more in the link: https://www.redhat.com/en/topics/devops/what-is-devsecops", '\n                        May 6-9, 2024\n                    you can read more in the link: https://www.redhat.com/en/summit?intcmp=7015Y000003sv6QQAQ', '\n                        May 6-9, 2024\n                    you can read more in the link: https://www.redhat.com/en/summit', 'Securely manage and scale deployments at the edge.you can read more in the link: https://www.redhat.com/en/topics', 'REST and SOAP are 2 different approaches to online data transmission. Specifically, both define how to build application programming interfaces (APIs), which allow data to be communicated between web applications. Representational state transfer (REST) is a set of architectu

In [149]:
#saveIndex(index)
db_collection_name = "ChatBotPatterns"
chatbot_patterns = get_patterns_from_db(db_collection_name)

# Create a chatbot
chatbot = Chat(chatbot_patterns, reflections)


SearchEngine

In [150]:
# Load and display a web page.
# Supports two types of HTML docs - 1. combined with css and js | 2. 3 separate files
def switchToScreen(css_filename_, html_filename_, js_filename_):
    clear_output(wait=True)

    if css_filename_ is None and js_filename_ is None:
        html_response = requests.get(html_filename_)
        html_content_ = html_response.text
        display(HTML(html_content_))
    else:
        css_response = requests.get(css_filename_)
        css_content_ = css_response.text
        display(HTML(f'<style>{css_content_}</style>'))

        html_response = requests.get(html_filename_)
        html_content_ = html_response.text
        display(HTML(html_content_))

        js_response = requests.get(js_filename_)
        js_content_ = js_response.text
        display(Javascript(js_content_))

In [151]:


def get_title_from_link(link):

  try:
        # Make an HTTP request
        response = requests.get(link)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the <h1> tag
            h1_tag = soup.find('h1')
            if h1_tag:
              return h1_tag.get_text()
            else:
              h2_tag = soup.find('h2')
              if h2_tag:
                return h2_tag.get_text()
        else:
            return f"Failed to fetch the page. Status code: {response.status_code}"

  except Exception as e:
        return f"An error occurred: {e}"

def check_title_list(lst):
  seen = {}
  result = []
  for word in lst:
    if word not in seen:
      seen[word] = 1
      result.append(word)
    else:
      seen[word] += 1
      result.append(f"{word}_{seen[word] - 1}")
  return result


def get_titles_links_of_search(search_string):
  query_counter = 0
  titles = list()
  links = set()
  search_string = search_string.split()
  # trimming and steaming algorithem to search terms exactly as they are in the DB
  search_string = remove_stop_words(search_string)
  search_string = trim_words(search_string)
  search_string = apply_stemming(search_string)
  for word in index.keys():
    if word in search_string:
      for i in range(min(10, len(index[word]))):
        curr_link = getLinkFromString(word, i)
        query_counter += index[word][curr_link]['counter']
        if curr_link not in links:
          titles.append(get_title_from_link(curr_link))
        links.add(curr_link)
  titles = check_title_list(titles)
  return titles, links, query_counter






In [204]:
def receive_search_string(search_string):
  original_search_string = search_string
  search_string = search_string.lower()
  html_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/results.html"
  js_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/script.js"
  switchToScreen(css_filename, html_filename, js_filename)

  js_code = f"""
    <script>
      document.getElementById("searchbartext").value = "{original_search_string}";
    </script>
    """

  titles, links, times_appeared = get_titles_links_of_search(search_string)

  # Convert Python arrays to JavaScript arrays
  titles_js_array = ', '.join(f'"{title}"' for title in titles)
  links_js_array = ', '.join(f'"{link}"' for link in links)
  times_appeared_str = str(times_appeared)

  js_code_2 = f"""
    <script>
      renderResults_title_link([ {titles_js_array} ], [ {links_js_array} ], [{times_appeared_str}]);
    </script>
  """

  display(HTML(js_code))
  display(HTML(js_code_2))

# Register the function to be called from JavaScript
output.register_callback('notebook.receive_search_string', receive_search_string)

def open_chatbot_page():
  html_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/chatbot.html"
  switchToScreen(css_filename, html_filename, js_filename)

# Register the function to be called from JavaScript
output.register_callback('notebook.open_chatbot_page', open_chatbot_page)

def receive_question_chatbot(question):
  response = chatbot.respond(question)
  if response is None:
    response = "Sorry, i dont have response for your answer :/"
  js_code = f"""
      <script>
        ChatbotResponse("{response}");
      </script>
    """
  #print(response)
  display(HTML(js_code))

# Register the function to be called from JavaScript
output.register_callback('notebook.receive_question_chatbot', receive_question_chatbot)

def back_to_home_page():
  html_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/index.html"
  switchToScreen(css_filename, html_filename, js_filename)

# Register the function to be called from JavaScript
output.register_callback('notebook.back_to_home_page', back_to_home_page)

css_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/Styling/style.css"
js_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/script.js"
html_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/index.html"

switchToScreen(css_filename, html_filename, js_filename)


In [153]:
def open_indexeditor_page():
  html_filename = "https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/admin.html"
  switchToScreen(None, html_filename, None)

# Register the function to be called from JavaScript
output.register_callback('notebook.open_indexeditor_page', open_indexeditor_page)

def open_statistics_screen():
  html_filename = "https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/statistics.html"
  switchToScreen(None, html_filename, None)

# Register the function to be called from JavaScript
output.register_callback('notebook.open_statistics_screen', open_statistics_screen)

def back_to_admin():
  html_filename = "https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/admin.html"
  switchToScreen(None, html_filename, None)

# Register the function to be called from JavaScript
output.register_callback('notebook.back_to_admin', back_to_admin)


In [154]:
def switchToScreen(css_filename_, html_filename_, js_filename_):
    clear_output(wait=True)

    if css_filename_ is None and js_filename_ is None:
        html_response = requests.get(html_filename_)
        html_content_ = html_response.text
        display(HTML(html_content_))
    else:
        css_response = requests.get(css_filename_)
        css_content_ = css_response.text
        display(HTML(f'<style>{css_content_}</style>'))

        html_response = requests.get(html_filename_)
        html_content_ = html_response.text
        display(HTML(html_content_))

        js_response = requests.get(js_filename_)
        js_content_ = js_response.text
        display(Javascript(js_content_))

In [186]:
def get_indexes_keys():
  indexes = list(index.keys())
  indexesArray = []
  for idx in indexes:
      indexesArray.append(idx)
  # Convert Python arrays to JavaScript arrays
  indexesArrayJS = ', '.join(f'"{indx}"' for indx in indexes)

  js_code = f"""
    <script>
      populateFirstCombo([ {indexesArrayJS} ]);
    </script>
    """
  display(HTML(js_code))

# Register the function to be called from JavaScript
output.register_callback('notebook.get_indexes_keys', get_indexes_keys)

def get_indexes_values(indexes):
    url = list(index[indexes].keys())
    # Convert Python arrays to JavaScript arrays
    urlArray = ', '.join(f'"{val}"' for val in url)
    js_code = f"""
      <script>
        populateSecondCombo([ {urlArray} ]);
      </script>
      """
    display(HTML(js_code))
# Register the function to be called from JavaScript
output.register_callback('notebook.get_indexes_values', get_indexes_values)

In [156]:
def get_random_indexes_with_counters():
    # Randomly select 5 indexes
    random_indexes = random.sample(list(index.keys()), 5)

    indexes = []
    counters = []

    # Calculate sum of counters for each selected index
    for idx in random_indexes:
        index_sum = sum(entry['counter'] for entry in index[idx].values())
        indexes.append(idx)
        counters.append(index_sum)

     # Convert Python arrays to JavaScript arrays
    indexesArray = ', '.join(f'"{indx}"' for indx in indexes)
    countersArray = ', '.join(f'"{counter}"' for counter in counters)

    js_code = f"""
    <script>
      generateGraph([ {indexesArray} ], [ {countersArray} ]);
    </script>
    """
    display(HTML(js_code))

# Register the function to be called from JavaScript
output.register_callback('notebook.get_random_indexes_with_counters', get_random_indexes_with_counters)

In [195]:
# Initialize Firebase Admin SDK
cred = credentials.Certificate("/content/CloudComputing-tirgul/Project/Json/sak.json")
if not firebase_admin._apps:
  firebase_admin.initialize_app(cred)
db = firestore.client()

# Define the function to update or add a document
def update_or_add_statistic(statistic_name):
    statistics_ref = db.collection("statisticsPanthers").document(statistic_name)
    doc = statistics_ref.get()
    if doc.exists:
        # Document exists, update the counter
        new_value = doc.to_dict().get("counter", 0) + 1
        statistics_ref.update({"counter": new_value})
    else:
        # Document doesn't exist, create a new one
        statistics_ref.set({"counter": 1})

In [158]:
# Function to get key with minimum counter
def get_key_with_min_counter():
    statistics_ref = db.collection("statisticsPanthers")

    query = statistics_ref.order_by("counter")

    # Get the first document (minimum counter)
    min_doc = query.limit(1).get()

    for doc in min_doc:
        min_key = doc.id
        min_counter = doc.to_dict()["counter"]

    return min_key, min_counter

# Function to get key with maximum counter
def get_key_with_max_counter():
    statistics_ref = db.collection("statisticsPanthers")

    query = statistics_ref.order_by("counter", direction=firestore.Query.DESCENDING)

    # Get the first document (maximum counter)
    max_doc = query.limit(1).get()

    for doc in max_doc:
        max_key = doc.id
        max_counter = doc.to_dict()["counter"]

    return max_key, max_counter

# Function to get keys with min and max counters
def get_min_max_keys():
    min_key, min_counter = get_key_with_min_counter()
    max_key, max_counter = get_key_with_max_counter()
    # Convert counters to strings
    min_counter_str = str(min_counter)
    max_counter_str = str(max_counter)
    js_code =  f"""
    <script>
      updateStatisticsBlock("{min_key}", "{min_counter_str}", "{max_key}", "{max_counter_str}");
    </script>
    """
    display(HTML(js_code))

# Register the function to be called from JavaScript
output.register_callback('notebook.get_min_max_keys', get_min_max_keys)

In [199]:
def delete_index_from_db(index_to_delete):
    collection_ref = db.collection("PantherIndex")
    docs = collection_ref.get()
    for doc in docs:
        if doc.to_dict()['term'] == index_to_delete:
            doc.reference.delete()
            index = get_index_from_db(db)
            break
output.register_callback('notebook.delete_index_from_db', delete_index_from_db)

In [201]:
def delete_link_from_db(link_index, link_to_delete):
    collection_ref = db.collection("PantherIndex")
    docs = collection_ref.get()
    for doc in docs:
        doc_dict = doc.to_dict()
        if "DocId" in doc_dict:
            doc_id_links = doc_dict["DocId"]
            if link_to_delete in doc_id_links:
                doc_id_links.pop(link_to_delete)
                doc.reference.update({"DocId": doc_id_links})
                index = get_index_from_db(db)
                break
output.register_callback('notebook.delete_link_from_db', delete_link_from_db)

In [205]:
print(get_index_from_db(db))

{'triallearn': {'https://www.redhat.com/en/blog?f[0]=taxonomy_topic_tid:27031#rhdc-search-listing': {'counter': 2, 'id': 23}, 'https://www.redhat.com/en/blog?f[0]=taxonomy_topic_tid:9001#rhdc-search-listing': {'counter': 2, 'id': 17}, 'https://www.redhat.com/en/blog?f[0]=taxonomy_topic_tid:75501#rhdc-search-listing': {'counter': 2, 'id': 12}, 'https://www.redhat.com/en/success-stories/verizon': {'counter': 2, 'id': 19}, 'https://www.redhat.com/en/topics/devops/what-is-devsecops': {'counter': 2, 'id': 26}, 'https://www.redhat.com/en/blog?f[0]=taxonomy_topic_tid:27061#rhdc-search-listing': {'counter': 2, 'id': 22}, 'https://www.redhat.com/en/topics/security/container-security': {'counter': 2, 'id': 24}, 'https://www.redhat.com/en/topics/integration/whats-the-difference-between-soap-rest': {'id': 20, 'counter': 2}, 'https://www.redhat.com/en/solutions': {'id': 5, 'counter': 2}, 'https://www.redhat.com/en/topics/security/api-security': {'id': 25, 'counter': 2}, 'https://www.redhat.com/en/b